# Operating System
1. Creating new processes:
    * fork() : create new process [Unix Based]
    * exec() : Program loading
    * PCB(Process Control Block) : meta data of the Process
    * <img src="images/fork1.png" width="30%" height="30%" />
    * When sys_fork() is called it creates a new process with the exact code as the parent but with different PID.
    * Parent code will have a `PID of the child` and for the `child process PID == 0.`
    * Execution is resumed in both the processes.
2. Virtual Memory;
    * <img src="images/fork2.png" width="40%" height="40%" />
3. exec():
    * Runs a program present in the file system (that is present in a correct executable format)
    * Creates a new process and the Child code in the Parent code will never be executed after the exec()
    * <img src="images/exec1.png" width="40%" height="40%" />
    * When exec() is called `PID remains the same` along with the file description.
4. File Descriptors:
    * fopen(),fread(), fwrite() - in C are all buffer IO and in turn makes a system call say open(), read() and write() etc.
    * <img src="images/fd1.png" width="40%" height="40%" />
    * The table is in kernel and the only way to access it is using 'fd' below.
    * First 3 slots are booked for console.
    * Accessing FD : fd = open("fodd.txt",O_RDONLY) | N = read(fd, buf, 100)
    * Code <redirect.c>:
        * close(1); // clears the stdout entry in FD
        * #include "kernel/fcntl.h" // to get the macros for the kernel
        * fd = open("hello.out", O_WRONLY| O_CREATE);
        * Read contents of the file, fd = open("hello.out",O_RDONLY); char buf[128]; n = read(fd, buf, 128); buf[n] = '\0';
5. Pipes:
    * A circular buffer to read and write the data into it.
    * Buffer is allocated in the Kernel.
    * Once all the write ends are closed then the read will unblock and continue.
    * <img src="images/pipe1.png" width="40%" height="40%" />
    * p(0) -> read end and p(1) -> write end
    * Code:
        * Important to do `pipe(p)` before fork so that the child and the parent has the pipe entry in the descriptor.
        * <img src="images/pipe2.png" />
        * Setting up > ls | wc 
        * <img src="images/pipe3.png" />
        * <img src="images/pipe4.png" width="40%" height="40%" />
          
6. Adding a Sys Call:
    * Tell us how many free pages are left in the kernel?  Ref `kalloc.c`, `defs.h` and search for // kalloc.c, then changes in `main.c` -> `syscall.h` and add .....(6:18) -> `syscall.c` and add entry ... -> `sysproc.c` ...
    * User side:
        * `user.h` add prototye -> `usys.pl` ...  -> Then create a user program to chech the sys call.
    * Code :
        * <img src="images/Sys1.png" width="75%" height="75%" />